In [22]:
# 사용한 라이브러리
import os
import json
import cv2
import numpy as np
import random

# 학습 데이터의 경로(이미지와 레이블)
image_dirs = [r".\original_data\image"]
label_dirs = [r".\original_data\label"] 

output_dir = r".\preprocessing_data"  # 저장 경로

In [23]:
# 이미지 전처리 함수
def preprocess_image(image, operations):
    # 그레이스케일 변환, 흑백 이미지로 변환
    if 'gray_scale' in operations:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 임계값 처리
    if 'thresholding' in operations:
        _, image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)
        
    if 'adaptive_thresholding' in operations:
        block_size, constant = 9, 3
        image = cv2.adaptiveThreshold(
            image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size, constant
            )
    
    if 'otsu_thresholding' in operations:
        _, image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # 블러링
    if 'blur' in operations:
        image = cv2.GaussianBlur(image, (5, 5), 0)
    
    # 투시 변환
    if 'perspective_transform_augmentation' in operations:
        # 원본 이미지의 크기 얻기
        height, width = image.shape[:2]
        
        # 변환 전 이미지의 꼭짓점을 변수에 할당
        pts1 = np.float32([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]])
        
        # 각각 이미지 내에서 임의의 좌표를 설정
        # distortion_factor: 이동 비율 
        distortion_factor=0.2
        pts2 = np.float32([
            [random.randint(0, int(width * distortion_factor)),
            random.randint(0, int(height * distortion_factor))],  # 왼쪽 상단
            [random.randint(int(width * (1-distortion_factor)), width),
            random.randint(0, int(height * distortion_factor))],  # 오른쪽 상단
            [random.randint(int(width * (1-distortion_factor)), width),
            random.randint(int(height * (1-distortion_factor)), height)],  # 오른쪽 하단
            [random.randint(0, int(width * distortion_factor)),
            random.randint(int(height * (1-distortion_factor)), height)]  # 왼쪽 하단
        ])
        
        # 원근 변환 행렬 계산
        mtrx = cv2.getPerspectiveTransform(pts1, pts2)
        
        # 원근 변환 적용 (이미지를 정해진 좌표로 변환)
        # 출력 이미지 크기를 원본보다 크게 설정 (예: 1.15배)
        new_width = int(width * (1.0 + distortion_factor))
        new_height = int(height * (1.0 + distortion_factor))
        
        # 변환된 이미지를 새로운 크기로 변환
        transformed_image = cv2.warpPerspective(image, mtrx, (new_width, new_height))
        
        # 출력된 이미지에서 원하는 영역만 잘라냄 (기존 이미지 크기만큼)
        start_x = (new_width - width) // 2
        start_y = (new_height - height) // 2
        
        # 변환된 이미지의 중앙에 기존 이미지 크기를 맞추어 자르기
        image = transformed_image[0:start_y + height, 0:start_x + width]

    return image

# 사용자 정의 전처리 작업 리스트
preprocessing_operations = ['gray_scale', 'adaptive_thresholding', 'blur']

In [ ]:
# 이미지 파일 이름 카운터
file_count = 1  
# 출력 디렉토리 생성
os.makedirs(output_dir, exist_ok=True)

for index, label_dir in enumerate(label_dirs):
    output_file = os.path.join(output_dir, "train.txt")
    Type = "train"
    
    with open(output_file, "w", encoding="utf-8") as label_file:
        for json_file in os.listdir(label_dir):
            if json_file.endswith(".json"):
                json_path = os.path.join(label_dir, json_file)
                
                # JSON 파일 읽기
                with open(json_path, "r", encoding="utf-8") as f:
                    data = json.load(f)
                
                # 이미지 파일 로드
                image_id = data["Images"]["identifier"]
                image_type = data["Images"]["type"]
                image_path = os.path.join(image_dirs[index], f"{image_id}.{image_type}")
                image = cv2.imread(image_path)
                
                if image is None:
                    print(f"Failed to load image: {image_path}")
                    continue
                
                # Bbox 데이터 처리
                for bbox in data.get("Bbox", []):
                    x_coords = bbox.get("x", [])
                    y_coords = bbox.get("y", [])
                    text = bbox.get("data", "").strip()
                    
                    print(text)
                    # 텍스트 저장
                    if text:
                        label_file.write(f"{Type}/word{file_count}.png\t{text}\n")
                    
                    # 부분 이미지 저장
                    if len(x_coords) == 4 and len(y_coords) == 4:
                        x1, y1 = min(x_coords), min(y_coords)
                        x2, y2 = max(x_coords), max(y_coords)
                        
                        if x2 > x1 and y2 > y1:  # 유효한 바운딩 박스인지 확인
                            sub_image = image[y1:y2, x1:x2]
                            
                            # 이미지 저장 경로 설정
                            output_image_dir = os.path.join(output_dir, Type)
                            os.makedirs(output_image_dir, exist_ok=True)  # Type별 서브디렉토리 생성
                            
                            # 전처리 전 이미지 저장
                            output_path = os.path.join(output_image_dir, f"word{file_count}_ori.png")
                            cv2.imwrite(output_path, sub_image)
                            
                            # 전처리 후 이미지 저장
                            output_path = os.path.join(output_image_dir, f"word{file_count}_pre.png")
                            sub_image = preprocess_image(sub_image, preprocessing_operations)
                            cv2.imwrite(output_path, sub_image)
                            
                            print(f"Saved: {output_path}")
                            file_count += 1
                        else:
                            print(f"Invalid bounding box in {json_path}: {x_coords}, {y_coords}")


지남에
Saved: .\preprocessing_data\train\word1_pre.png
시간이
Saved: .\preprocessing_data\train\word2_pre.png
세상이
Saved: .\preprocessing_data\train\word3_pre.png
믿는다면
Saved: .\preprocessing_data\train\word4_pre.png
증거를
Saved: .\preprocessing_data\train\word5_pre.png
것이다
Saved: .\preprocessing_data\train\word6_pre.png
해도
Saved: .\preprocessing_data\train\word7_pre.png
보아라
Saved: .\preprocessing_data\train\word8_pre.png
열렬@
Saved: .\preprocessing_data\train\word9_pre.png
편을
Saved: .\preprocessing_data\train\word10_pre.png
@은
Saved: .\preprocessing_data\train\word11_pre.png
1
Saved: .\preprocessing_data\train\word12_pre.png
변@에
Saved: .\preprocessing_data\train\word13_pre.png
대한
Saved: .\preprocessing_data\train\word14_pre.png
저항력이
Saved: .\preprocessing_data\train\word15_pre.png
매우
Saved: .\preprocessing_data\train\word16_pre.png
강@다
Saved: .\preprocessing_data\train\word17_pre.png
2
Saved: .\preprocessing_data\train\word18_pre.png
일반적인
Saved: .\preprocessing_data\train\word19_pre.png
믿음은
Save